<a href="https://colab.research.google.com/github/mgntprg/DL_Book/blob/master/blackboard_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

In [0]:
!pip uninstall tensorflow

In [0]:
!pip install tensorflow==2.0.0

In [0]:
!pip install -q git+https://github.com/tensorflow/examples.git

In [0]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow_examples.models.pix2pix import pix2pix

In [0]:
def normalize(input_image, input_mask):
  input_image -= tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

We need to modify the training data such that we can make the result predictable by segmentation. There are only 2 classes which cna be predicted by segmentation, so it would be a binary problem.

Shape of the mask would be (x, x, 1) with the final dimension used to represent the number of class outputs

In [0]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (450, 600))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (450, 600))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)
  
  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [0]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

Create a function which generates a target mask from an input image

In [0]:
# 1 if white, 2 if black (this is shifted to 0 and 1 in normalize() )
def make_mask(img):
  mask = np.empty((img.shape[0], img.shape[1], 1))
  for i, row in enumerate(img):
    # row is a list of pixels for the image in the particular row
    for j, col in enumerate(row):
      if col[0] < 125:
        mask[i][j][0] = 1
      else: 
        mask[i][j][0] = 2
  return mask

In [0]:
import os

paths = []
 
input_dir = 'gdrive/My Drive/ML/Projects/Blackboard_effect/images/input'
output_dir = 'gdrive/My Drive/ML/Projects/Blackboard_effect/images/output'

X = []
y = []

for file_name in os.listdir(input_dir):
  
  img = cv2.imread(os.path.join(input_dir, file_name))   # reads an image in the BGR format
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # BGR -> RGB
  
  img_filter = cv2.imread(os.path.join(output_dir, file_name)) 
  mask = make_mask(img_filter)

  X.append([img])
  y.append([mask])
  display([img, mask])

In [0]:
OUTPUT_CHANNELS = 1 # Black or white

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[224, 224, 3], include_top=False)

# use activations of these layers
layer_names = [
    'block_1_expand_relu',   # 112x112
    'block_3_expand_relu',   # 56x56
    'block_6_expand_relu',   # 28x28
    'block_13_expand_relu',  # 14x14
    'block_16_project',      # 7x7
]

layers = [base_model.get_layer(name).output for name in layer_names]

down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
down_stack.trainable = False

In [0]:
# first param of upsample is pixels
up_stack = [
    pix2pix.upsample(512, 3), # 4x4 -> 8x8
    pix2pix.upsample(256, 3), # 8x8 -> 16x16
    pix2pix.upsample(128, 3), # 16x16 -> 32x32
    pix2pix.upsample(64, 3) # 32x32 -> 64x64
]

In [0]:
def unet_model(output_channels):
  last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2, padding='same', activation='sigmoid') # sigmoid since only 2

  inputs = tf.keras.layers.Input(shape=[224, 224, 3])
  x = inputs

  skips = down_stack(x)
  x = skips[-1] # downsize all the way
  skips = reversed(skips[:-1]) # reverse layer order, for residual (adds info from the encoder)

  # upsampling and establish skip connection
  for up, skip in zip(up_stack, skips):
    x = up(x) # apply upsampling 
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip]) # establish skip connections
  
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [0]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
EPOCHS = 1000

In [0]:
print('Mask shape:', mask.shape)
for i in range(EPOCHS):
  model_history = model.fit(X, y, epochs=1)

In [0]:
input_file = r'gdrive/My Drive/ML/Projects/Blackboard_effect/14.5.jpg'
img = cv2.imread(input_file).astype(np.float32)  # reads an image in the BGR format
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # BGR -> RGB

In [0]:
pred = model.predict(np.array([img]))

In [0]:
for mask in pred:
  print(mask.shape)
  color_img = mask * 255.0
  color_img = cv2.cvtColor(color_img, cv2.COLOR_GRAY2RGB)
  display([color_img])

In [0]:
  print(mask * 255)
mask[0][0][0]